In [1]:
import sympy as sp
import numpy as np

In [2]:
import sympy.vector

In [3]:
from IPython.display import display

In [4]:
N = sp.vector.CoordSys3D('N')

In [5]:
class Node:
    def __init__(self, tup):
        x, y, z = tup
        self.position = x*N.i + y*N.j + z*N.k
        
    def __sub__(self, other):
        return self.position - other.position

In [6]:
class Force:
    def __init__(self, input1, input2):
        """
        input1 must be a starting point/node
        input2 can be another node to be an end point, or force vector
        """
        assert type(input1) == Node
        self.start_position = input1.position
        self.unique_symbols = set()
        
        if type(input2) == Node:
            diff_vector = input2 - input1
            self.force_vector = diff_vector / np.linalg.norm(diff_vector)
        elif type(input2) == sp.vector.vector.BaseVector:
            self.force_vector = input2
        else:
            fx, fy, fz = input2
            mx = my = mz = 1
            if type(fx) == tuple:
                mx, fx = fx
            if type(fx) == str:
                fx = sp.Symbol(fx)
                self.unique_symbols.add(fx)
                
            if type(fy) == tuple:
                my, fy = fy
            if type(fy) == str:
                fy = sp.Symbol(fy)
                self.unique_symbols.add(fy)
                
            if type(fz) == tuple:
                mz, fz = fz
            if type(fz) == str:
                fz = sp.Symbol(fz)
                self.unique_symbols.add(fz)
                
            self.force_vector = mx*fx*N.i + my*fy*N.j + mz*fz*N.k
        self.torque_vector = sp.vector.cross(self.start_position, self.force_vector)

In [7]:
def solve_sum_zero(forces_eq_0, exclude_list = [], additional_symbols = [], additional_eqs = [], only_torque = False):
    unique_symbols = set()
    sym_zero = 0*(N.i+N.j+N.k)
    force_eq_0 = 0*(N.i+N.j+N.k)
    torque_eq_0 = 0*(N.i+N.j+N.k)
    for i in forces_eq_0:
        unique_symbols |= i.unique_symbols
        force_eq_0 += i.force_vector
        torque_eq_0 += i.torque_vector
    for i in exclude_list:
        ii = sp.Symbol(i)
        if ii in unique_symbols:
            unique_symbols.remove(ii)
    symbols = list(unique_symbols)
    equations = []
    if not only_torque:
        for cp in force_eq_0.components:
            equations.append(sp.Eq(force_eq_0.components[cp], 0))
    for cp in torque_eq_0.components:
        equations.append(sp.Eq(torque_eq_0.components[cp], 0))
    equations += additional_eqs
    symbols += additional_symbols
    solved = sp.solve(equations, symbols)
    if type(solved) == list:
        print("Cannot find answer")
        print(equations)
    else:
        for k in sorted(list(solved.keys()), key=lambda x: str(x)):
            display(sp.Eq(k, solved[k]))

# Homework 1

In [8]:
x = sp.Symbol('x')
y = sp.Symbol('y')
z = sp.Symbol('z')
F = sp.Symbol('F')
A = Node((0, y, 0))
B = Node((0, 0, z))
C = Node((x, 0, 0))
F_A = Force(A, (F, 0, 0))
F_B = Force(B, (0, F, 0))
F_C = Force(C, (0, 0, F))

px, py, pz = sp.symbols('P_x P_y P_z')
# Balance node
P = Node((px, py, pz))
F_P = Force(P, (-F, -F, -F))

In [9]:
F_P.torque_vector

(-F*P_y + F*P_z)*N.i + (F*P_x - F*P_z)*N.j + (-F*P_x + F*P_y)*N.k

In [10]:
solve_sum_zero([F_A, F_B, F_C, F_P], [], [px, py, pz], [sp.Eq(x+y+z,0)], False) #, [sp.Eq(fpx**2+fpy**2+fpz**2, F**2)])

Cannot find answer
[Eq(-F*P_x + F*P_y - F*y, 0), Eq(-F*P_y + F*P_z - F*z, 0), Eq(F*P_x - F*P_z - F*x, 0), Eq(x + y + z, 0)]


We cannot find the answer that results in "balanced" torque.

# Homework 2

In [11]:
l = 1
a = 0.5
theta = np.deg2rad(30)
alpha = np.deg2rad(30)
mag_F_F = 10

In [12]:
A = Node((0, 0, -l))
B = Node((0, 0, l))
A_unknown = Node((0, 0, -sp.Symbol('l')))
B_unknown = Node((0, 0, sp.Symbol('l')))
D = Node((-a*sp.sin(theta), -a*sp.cos(theta), 0))
D_unknown = Node((-sp.Symbol('a')*sp.sin(sp.Symbol('\\theta')), -sp.Symbol('a')*sp.cos(sp.Symbol('\\theta')), 0))
E = Node((a, 0, 0))

In [13]:
F_A = Force(A, ('F_{Ax}', 'F_{Ay}', 'F_{Az}'))
F_B = Force(B, ('F_{Bx}', 'F_{By}', 0))
F_A_unknown = Force(A_unknown, ('F_{Ax}', 'F_{Ay}', 'F_{Az}'))
F_B_unknown = Force(B_unknown, ('F_{Bx}', 'F_{By}', 0))
F_F_known = Force(D, (0, -sp.sin(alpha)*mag_F_F, -sp.cos(alpha) * mag_F_F))
F_F_unknown = Force(D, (0, (-sp.sin(alpha), 'F_F'),  (-sp.cos(alpha), 'F_F')))
F_F_all_unknown = Force(D_unknown, (0, (-sp.sin(sp.Symbol('\\alpha')), 'F_F'),  (-sp.cos(sp.Symbol('\\alpha')), 'F_F')))
F_P = Force(E, (0, 'F_P', 0))

## All known values

In [14]:
solve_sum_zero([F_A, F_B, F_F_known, F_P])

Eq(F_P, -2.5)

Eq(F_{Ax}, -1.08253175473055)

Eq(F_{Ay}, 1.875)

Eq(F_{Az}, 8.66025403784439)

Eq(F_{Bx}, 1.08253175473055)

Eq(F_{By}, 5.625)

## All known values, except $\vec{F_F}$

In [15]:
solve_sum_zero([F_A, F_B, F_F_unknown, F_P], ['F_F'])

Eq(F_P, -0.25*F_F)

Eq(F_{Ax}, -0.108253175473055*F_F)

Eq(F_{Ay}, 0.1875*F_F)

Eq(F_{Az}, 0.866025403784439*F_F)

Eq(F_{Bx}, 0.108253175473055*F_F)

Eq(F_{By}, 0.5625*F_F)

## All unknown

In [16]:
solve_sum_zero([F_A_unknown, F_B_unknown, F_F_all_unknown, F_P], ['a', '\\theta', 'F_F'])

Eq(F_P, -2.0*F_F*a*sin(\alpha)*sin(\theta))

Eq(F_{Ax}, -0.5*F_F*a*sin(\theta)*cos(\alpha)/l)

Eq(F_{Ay}, F_F*a*sin(\alpha)*sin(\theta) - 0.5*F_F*a*cos(\alpha)*cos(\theta)/l + 0.5*F_F*sin(\alpha))

Eq(F_{Az}, F_F*cos(\alpha))

Eq(F_{Bx}, 0.5*F_F*a*sin(\theta)*cos(\alpha)/l)

Eq(F_{By}, F_F*a*sin(\alpha)*sin(\theta) + 0.5*F_F*a*cos(\alpha)*cos(\theta)/l + 0.5*F_F*sin(\alpha))